In [ ]:
import tableauserverclient as TSC
import pandas as pd
import psycopg2

# Tableau Server URL, Site ID, Personal Access Token Name and Token Secret code
server_url = 'put_your_server_url_here'
site = 'put_your_site_name_here'
mytoken_name = 'put_your_token_name_here'
mytoken_secret = 'put_your_token_secret_code_here'



def getTableauServerUsers(server_url, mytoken_name, mytoken_secret, site=site):

    # Import the Tableau Server Client (TSC) library
    server = TSC.Server(server_url, use_server_version=True)

    # Authenticate with the Tableau Server using a Personal Access Token
    tableau_auth = TSC.PersonalAccessTokenAuth(token_name=mytoken_name, personal_access_token=mytoken_secret, site_id=site)

    with server.auth.sign_in_with_personal_access_token(tableau_auth):
        print('[Logged in successfully to {}]'.format(server_url))

        print('[Loading users...]')

        # Create an empty list to store the user information
        tableauUsers = []

        # Use the TSC Pager to iterate through all users
        for user in TSC.Pager(server.users.get):

            # Populate the groups for each user
            server.users.populate_groups(user)

            # Get the name of each group the user belongs to
            groups = [group.name for group in user.groups]

            # Append the user information to the tableauUsers list
            tableauUsers.append(
                (
                    user.id,
                    user.name.lower().strip() if user.name and isinstance(user.name,str) else None,
                    user.email.lower().strip() if user.email and isinstance(user.email,str) else None,
                    user.domain_name,
                    user.fullname,
                    user.site_role,
                    user.last_login,
                    groups
                )
            )
        print('[Tableau users loaded]')
        # Return the list of users
        return tableauUsers

result = getTableauServerUsers(server_url, mytoken_name, mytoken_secret, site=site)



def create_and_explode_df(result):

    # Create a dataframe from the result and rename columns
    columns = ['id', 'username', 'email', 'domain_name', 'fullname', 'site_role', 'last_login', 'groups']
    df = pd.DataFrame(result, columns=columns)

    # Explode the 'groups' field into different rows
    df = df.explode('groups')

    return df

df = create_and_explode_df(result)


# This function uses local PostgreSQL DB parameters
def insert_df_to_postgres(df, host='localhost', port=5432, dbname='postgres', user='postgres', password='postgres', table_name='tableau_users'):

    # Connect to the database
    conn = psycopg2.connect(host=host, port=port, dbname=dbname, user=user, password=password)
    cursor = conn.cursor()

    # Create a table to store the dataframe
    cursor.execute(f'''DROP TABLE IF EXISTS {table_name}; CREATE TABLE {table_name} (
                      id VARCHAR(255),
                      username VARCHAR(255),
                      email VARCHAR(255),
                      domain_name VARCHAR(255),
                      fullname VARCHAR(255),
                      site_role VARCHAR(255),
                      last_login TIMESTAMP,
                      groups VARCHAR(255)
                  )''')

    # Convert the 'last_login' column to a datetime data type
    df['last_login'] = pd.to_datetime(df['last_login'])

    # Replace any 'NaT' values in the 'last_login' column with NULL values
    df['last_login'] = df['last_login'].fillna(pd.NA)

    # Insert the dataframe into the table
    for i, row in df.iterrows():
        if pd.isnull(row['last_login']):
            row['last_login'] = None
        cursor.execute("INSERT INTO {} VALUES (%s, %s, %s, %s, %s, %s, %s, %s)".format(table_name), (
            row['id'], 
            row['username'], 
            row['email'], 
            row['domain_name'], 
            row['fullname'], 
            row['site_role'], 
            row['last_login'], 
            row['groups'])
            )

    # Commit the transaction
    conn.commit()
    
    # Select the count of rows from the table
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"{count} rows were saved to the {table_name} table.")

    # Close the cursor and connection
    cursor.close()
    conn.close()

df = create_and_explode_df(result)
insert_df_to_postgres(df, host='localhost', port=5432, dbname='postgres', user='postgres', password='postgres')



# Additionally save dataframe to the csv in Python directory
df.to_csv('tableau_users.csv', index=False)
